# La transformada continua Wavelet 
***

### Sinopsis

*__Datos:__ 3 segundos de registro EEG lóbulo occipital muestreados a 500 Hz bajo condiciones de relajación con los ojos cerrados.* <br>
*__Objetivo:__ Conocer, definir e implementar la transformada contínua wavelet para señales no estacionarias.* <br>
*__Herramientas:__ Wavelets, Transformada Contínua Wavelet.* <br>

### Introducción

Cuando se busca realizar algún método de análisis y/o procesamiento de señales, un requisito usual en éstos métodos, tales como la Transformada de Fourier, es la estacionariedad. Ésta característica de ciertas señales describe que a lo largo de la serie de tiempo sus propiedades estadísticas no cambian a lo largodel tiempo, algo que muchas veces o inclusive nunca es logra en señales de orígen biológico como lo es el caso de la electroencefalografía. Por esta razón, en términos prácticos muchas veces se asume la estacionariedad en un término más flexible llamado "estacionariedad en sentido amplio", el cual requiere una media constante y una función de correlación invariante en el tiempo.  Sin embargo, para una señal con media distinta de cero el equivalente para esta condición en el dominio de la frecuencia es que cuente con un espectro invariante en el tiempo. Por este motivo, en la práctica para emplear los métodos de análisis y procesamiento convencionales es necesario implementarlos sobre segmentos de tiempo cortos donde sea posible considerar a la señal como estacionaria estricta o en sentido amplio. La duración  para estos segmentos varía y depende del tipo de señal que se busque analizar, en el caso de resitros de EEG los segmentos pueden ser hasta de 20 segundos de duración (Cassani & Falk, 2019). De aquí surge la necesidad de contar con una representación  capaz de regristrar los cambios en el contenido de la frecuencia a lo largo del tiempo para llevar a cabo un análisis frecuencial sobre señales no estacionarias, es decir, se requiere de la construcción de una representación espectro-temporal la cuál puede ser obtenida mediante distintos métodos como lo son: 
<br>

    - Transformada Corta de Fourier (STFT)
    - Transformada Hilbert  (HT)
    - Transformada Continua Wavelet (CWT)

### Dominio Tiempo y Frecuencia

Un elemento fundamental en el análisis y procesamiento de señales es la capacidad de representar señales en el dominio temporal y frecuencial mediante transformaciones matemáticas reversibles. Si bien existen muchos métodos para realizar estas transformaciones, el más utilizado es la transformada de Fourier y la transformada de Fourier inversa, que realizan la transformación tiempo-a-frecuencia y frecuencia-a-tiempo respectivamente. Matemáticamente se definen como sigue

<script type="text/x-mathjax-config">
MathJax.Hub.Config({
tex2jax: {
inlineMath: [['$','$'], ['\\(','\\)']],
processEscapes: true},
jax: ["input/TeX","input/MathML","input/AsciiMath","output/CommonHTML"],
extensions: ["tex2jax.js","mml2jax.js","asciimath2jax.js","MathMenu.js","MathZoom.js","AssistiveMML.js", "[Contrib]/a11y/accessibility-menu.js"],
TeX: {
extensions: ["AMSmath.js","AMSsymbols.js","noErrors.js","noUndefined.js"],
equationNumbers: {
autoNumber: "AMS"
}
}
});
</script>

<p align="center">
    $$ X(\omega) = \mathcal{F}\{x(t)\} = \frac{1}{\sqrt{2\pi}}\int^{\infty}_{-\infty} x(t) e^{-i2\pi\omega t}dt$$
    <br>
    $$ x(t) = \mathcal{F}^{-1}\{X(\omega)\} =\frac{1}{\sqrt{2\pi}} \int^{\infty}_{-\infty} X(\omega) e^{i2\pi\omega t}d\omega$$
</p>

Donde $\mathcal{F}\{\cdot\}$ y $\mathcal{F}^{-1}\{ \cdot\}$ denotan la transformada de Fourier (TF) y la transformada inversa (TFI) respectivamente; $x(t)$ es la señal en el dominio del tiempo; $X(\omega)$ es una función compleja llamado espectro de $x(t)$ que bien puede escribirse como:

$$X(\omega) = ||X(\omega)||e^{i\theta(\omega)}$$

donde $||X(\omega)||$ es la amplitud espectral con una fase $\theta(\omega)$ asociada. En general, $||X(\omega)||$ representa la amplitud para dada frecuencia $\omega$, con la cual se obtiene una representación usual en el dominio de la frecuencia que es la densidad espectral de energía $||X(\omega)||^2$, la cual muestra la distribución de la energía de la señal para distintas frecuencias; es decir, el área bajo la curva $||X(\omega)||^2$ entre $f_0-df$ y $f_0+df$ representa la energía de la componente $f_0$ en Hz.

Ocurre un caso especial en el caso de TF para funciones $x(t)$ reales, en el cuál su espectro correspondiente presenta simetria Hermitiana, es decir las componentes espectrales para funciones negativas son equivalentes al complejo conjugado de las componentes para frecuencias positivas, es decir:

$$X(\omega) = X^{*}(-f) \Rightarrow ||X(\omega)|| = ||X^{*}(-f)|| $$

Como consecuencia, la representación del espectro de señales reales se realiza solo para frecuencias positivas, por consiguiente para conservar la correcta relación entre componentes frecuenciales y su respectiva amplitud $||X(\omega)||$ y potencia $||X(\omega)||^2$, se deben duplicar los valores. 

Una cualidad importante que es útil y sencilla para corroborar que se realice una correcta transformada de Fourier es mediante la "Relación de Parseval" la cual demuestra que la TF es una operación unitaria, es decir, la integral (o la suma en el caso discreto) del cuadrado de una función es igual a la integral (o suma) del cuadrado de su transformada. Matemáticamente se expresa como:

$$\int_{-\infty}^{\infty} ||x(t)||^2 dt = \int_{-\infty}^{\infty} ||X(\omega)||^2 d\omega$$

O en su versión discreta, donde $X[j]$ representa la TFD de $x[n]$ :

$$\sum^{N-1}_{n=0}|x[n]|^2 = \frac{1}{N}\sum^{N-1}_{n=0} |X[j]|^2 $$
Como ejercicio podemos emplear el código presentado en el módulo "La electroencefalografía" para obtener y graficar la TF y comprobar la Relación Parseval:

##### Importe de paqueterías

In [1]:
import pandas as pd  #Paq. p-manejo de archivos de datos
import scipy as sc  #Paq. p-análisis numérico 
import numpy as np  #Paq. p-cálculo numérico
import matplotlib   #Paq. p-graficación
import matplotlib.pyplot as plt
import os.path as os #Paq. p-despliegue de ruta

##### Importación de los datos

In [73]:
#Una variable para la dirección de nuestros archivos
path = os.abspath('C:/Users/Andrew S/Documents/Trabajo/Fisica Biomedica/8vo/Servicio Social/')
#Asociamos una variable al objeto DataFrame creado apartir del archivo extensión csv
data = pd.read_csv(path +'/Jenny.csv')
EEG = data['meditacion'] #Señal EEG en reposo

##### Relación de Parseval

A continuación realizamos el cálculo de la TF haciendo uso del algoritmo de FFT, posteriormente comprobamos la relación definiendo una función que realice la "integración", es decir la suma de los valores absolutos cuadrados i de la serie.

In [74]:
def Suma (x): #La función recibira una variable de tipo array
    suma = 0 #Definimos el valor inicial de la suma en 0
    for i in range(len(x)): #iniciamos un ciclo for durante el cual se realizará la operación
        suma += abs(x[i])**2 #iterativamente sumamos el valor anterior de la variable más el elemento i del array
    return suma

In [82]:
P1_time = Suma(EEG) #Obtenemos la suma del array 'x^2' y lo asignamos a una nueva variable
P2_frequency = 1/N * Suma(sc.fft(EEG)) #Realizamos el procedimiento análogo para el array del dominio de las frecuencias
P1_time // P2_frequency

<ipython-input-82-4e57879c4527>:2: DeprecationWarning: Using scipy.fft as a function is deprecated and will be removed in SciPy 1.5.0, use scipy.fft.fft instead.
  P2_frequency = 1/N * Suma(sc.fft(EEG)) #Realizamos el procedimiento análogo para el array del dominio de las frecuencias


0.0

### Operaciones fundamentales

#### La convolución

Una de las operaciones fundamentales sobre la que reposa la WCT, es la convolución, la cual podemos entender en palabras simples como un operador que toma dos funciones y regresa una tercera función que cuenta con los elementos de ambas funciones ya que representa la magnitud en la que se superpone la primera función y la segunda función invertida y trasladada. Matemáticamente en el espacio de funciones continuas se expresa para dadas funciones $f$ y $g$ como: 

$$ f(t)\star g(t) = \int_{-\infty}^{\infty} f(\tau)g(t-\tau) d\tau$$

### Descomposición Wavelet



### Wavelets - los kernels

### Transformada Contínua Wavelet